In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
# to use a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"

In [2]:
from mgtbench import AutoDetector, AutoExperiment
from mgtbench.loading.dataloader import load

In [3]:
'''
supported LLMs and detect categories:

categories = ['Physics', 'Medicine', 'Biology', 'Electrical_engineering', 'Computer_science', 'Literature', 'History', 'Education', 'Art', 'Law', 'Management', 'Philosophy', 'Economy', 'Math', 'Statistics', 'Chemistry']

llms = ['Moonshot', 'gpt35', 'Mixtral', 'Llama3', 'gpt-4omini']
'''
data_name = 'AITextDetect'
detectLLM = 'Llama3'
category = 'Art'

In [4]:
data = load(data_name, detectLLM, category)
# for demo
size = 50
data['train']['text'] = data['train']['text'][:size]
data['train']['label'] = data['train']['label'][:size]
data['test']['text'] = data['test']['text'][:size]
data['test']['label'] = data['test']['label'][:size]

loading human data
loading machine data
data loaded


parsing data: 100%|██████████| 8394/8394 [00:00<00:00, 16180.37it/s]


#### Model-based Detector

In [4]:
# local path to the model, or model name on huggingface
model_name_or_path = '/data1/models/distilbert-base-uncased'
metric = AutoDetector.from_detector_name('LM-D', 
                                            model_name_or_path=model_name_or_path)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /data1/models/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


length is set to 512


In [9]:
experiment = AutoExperiment.from_experiment_name('supervised',detector=[metric])
experiment.load_data(data)
config = {'need_finetune': True,
          'need_save': False,
          'epochs': 1, # for model-based detectors
          }
res = experiment.launch(**config)

print('==========')
print('train:', res[0].train)
print('test:', res[0].test)

/home/zhiyuan/miniconda3/envs/mgtbench2/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Calculate result for each data point
Running prediction of detector LM-D
False


Step,Training Loss


Fine-tune finished
Predict training data


100%|██████████| 100/100 [00:00<00:00, 170.22it/s]


Predict testing data


100%|██████████| 100/100 [00:00<00:00, 180.53it/s]

Run classification for results
train: Metric(acc=0.6, precision=0.7142857142857143, recall=0.21739130434782608, f1=0.3333333333333333, auc=0.698268921095008, conf_m=None)
test: Metric(acc=0.58, precision=0.6470588235294118, recall=0.23404255319148937, f1=0.34375, auc=0.6967081493376154, conf_m=None)


#### Metric-based Detector

In [10]:
# log-likelihood detector
model_name_or_path = '/data1/zzy/gpt2-medium'
metric = AutoDetector.from_detector_name('ll',
                                         model_name_or_path=model_name_or_path)

In [11]:
experiment = AutoExperiment.from_experiment_name('threshold',detector=[metric])
experiment.load_data(data)
res = experiment.launch()

print('==========')
print('train:', res[0].train)
print('test:', res[0].test)

Calculate result for each data point
Running prediction of detector ll
Predict training data


  3%|▎         | 3/100 [00:00<00:03, 24.84it/s]

100%|██████████| 100/100 [00:01<00:00, 56.52it/s]


Predict testing data


100%|██████████| 100/100 [00:01<00:00, 63.46it/s]

Run classification for results
train: Metric(acc=0.85, precision=0.8780487804878049, recall=0.782608695652174, f1=0.8275862068965517, auc=0.8945249597423511, conf_m=None)
test: Metric(acc=0.85, precision=0.8076923076923077, recall=0.8936170212765957, f1=0.8484848484848485, auc=0.9359694901645925, conf_m=None)


#### Fast-DetectGPT


In [7]:
scoring_model_name_or_path = '/data_sda/zhiyuan/models/gpt-neo-2.7B'
reference_model_name_or_path = '/data_sda/zhiyuan/models/gpt-j-6B'
fastDetectGPT = AutoDetector.from_detector_name('fast-detectGPT', 
                                            scoring_model_name_or_path=scoring_model_name_or_path,
                                            reference_model_name_or_path= reference_model_name_or_path
                                            )

Some weights of the model checkpoint at /data_sda/zhiyuan/models/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transforme

In [8]:
experiment = AutoExperiment.from_experiment_name('perturb', detector=[fastDetectGPT])
experiment.load_data(data)
res = experiment.launch()

print('==========')
print('train:', res[0].train)
print('test:', res[0].test)

Calculate result for each data point
Running prediction of detector fast-detectGPT
Predict training data


Detecting:   0%|          | 0/50 [00:00<?, ?it/s]

Detecting: 100%|██████████| 50/50 [00:11<00:00,  4.27it/s]


Predict testing data


Detecting: 100%|██████████| 50/50 [00:13<00:00,  3.80it/s]

Run classification for results
Finding best threshold for f1 score...
train: Metric(acc=0.82, precision=0.8214285714285714, recall=0.8518518518518519, f1=0.8363636363636363, auc=0.8631239935587762, conf_m=None)
test: Metric(acc=0.9, precision=0.8666666666666667, recall=0.9629629629629629, f1=0.9122807017543859, auc=0.9355877616747181, conf_m=None)


#### Binocolars Detector


In [9]:
observer_model_name_or_path = '/data_sda/zhiyuan/models/falcon-7b'
performer_model_name_or_path = '/data_sda/zhiyuan/models/falcon-7b-instruct'

binoculars = AutoDetector.from_detector_name('Binoculars', 
                                            observer_model_name_or_path=observer_model_name_or_path,
                                            performer_model_name_or_path= performer_model_name_or_path,
                                            max_length=1024,
                                            mode='low-fpr', # accuracy (f1) or low-fpr
                                            # 'default' or 'new', default is the threshold used in the paper, 'new' is the threshold calculated on the new training set
                                            threshold='new' 
                                            )

/data_sda/zhiyuan/models/falcon-7b /data_sda/zhiyuan/models/falcon-7b-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# threshold set for low-fpr
binoculars.change_mode('low-fpr')
experiment = AutoExperiment.from_experiment_name('threshold', detector=[binoculars])
experiment.load_data(data)
res = experiment.launch()
print('==========')
print('train:', res[0].train)
print('test:', res[0].test)

# threshold set for f1
binoculars.change_mode('accuracy')
experiment = AutoExperiment.from_experiment_name('threshold', detector=[binoculars])
experiment.load_data(data)
res = experiment.launch()
print('==========')
print('train:', res[0].train)
print('test:', res[0].test)

Calculate result for each data point
Running prediction of detector Binoculars
Predict training data


Detecting: 100%|██████████| 50/50 [00:14<00:00,  3.49it/s]


Predict testing data


Detecting: 100%|██████████| 50/50 [00:16<00:00,  3.03it/s]


Run classification for results
Finding best threshold for low-fpr...
train: Metric(acc=0.54, precision=0.54, recall=1.0, f1=0.7012987012987013, auc=0.9500805152979066, conf_m=None)
test: Metric(acc=0.54, precision=0.54, recall=1.0, f1=0.7012987012987013, auc=0.9420289855072463, conf_m=None)
Calculate result for each data point
Running prediction of detector Binoculars
Predict training data


Detecting: 100%|██████████| 50/50 [00:14<00:00,  3.54it/s]


Predict testing data


Detecting: 100%|██████████| 50/50 [00:16<00:00,  2.97it/s]

Run classification for results
Finding best threshold for accuracy...
train: Metric(acc=0.9, precision=0.9230769230769231, recall=0.8888888888888888, f1=0.9056603773584906, auc=0.9500805152979066, conf_m=None)
test: Metric(acc=0.84, precision=0.8518518518518519, recall=0.8518518518518519, f1=0.8518518518518519, auc=0.9420289855072463, conf_m=None)
